# License

In [8]:
# Copyright 2017 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

def installDeps():
    !pip install numpy scipy
    !pip install resampy tensorflow six
    !pip install youtube_dl
    !pip install ipywidgets
    !pip install pydub
    !pip install tqdm
    !pip install ffmpeg-python
    !apt-get install ffmpeg -y
    #!wget https://storage.googleapis.com/audioset/vggish_model.ckpt
    #!wget https://storage.googleapis.com/audioset/vggish_pca_params.npz
#installDeps()
#!python vggish_train_demo.py --num_batches 50 --train_vggish=False --checkpoint './vggish_model.ckpt'

from __future__ import print_function

from random import shuffle

import numpy as np
import tensorflow as tf
import os
import vggish_input
import vggish_params
import vggish_slim
from pydub import AudioSegment
from audioModel import predict, train
from audioInput import getLaughTracks, getNoise
from audioUtils import readFolder

def getModel(path):
    files = readFolder('model/%s' % path)
    if len(files) > 0:
        return '%s/%s' % (path, files[0])
    return None

def trainAndSaveAndPredict(test_data, model, number_of_classes = 2, number_of_samples = 1, epochs = 5, getData = getLaughTracks, log = True):
    model_name = '%s_%s' % (model, number_of_samples)
    def curriedGetSamples(shuf):
        return getData(number_of_samples = number_of_samples, shuf = shuf, log = log)
    print('model_name', model_name)
    preds = train(curriedGetSamples, number_of_classes, model_name = model_name, epochs = epochs)
    
    return predict(getModel('%s' % (model_name)), number_of_classes, test_data)

def printResults(preds, expected = None): 
    with tf.Graph().as_default(), tf.Session() as sess:
        print(preds)
        print(sess.run(tf.argmax(input=preds, axis=1))) 
        print('expected results', expected)

def trainForNoise(number_of_samples=5, epochs=5):
    print('training on noise, sin, and constant waves')
    (features, labels) = getNoise(shuf=False, number_of_samples = 2)
    preds = trainAndSaveAndPredict(features, 'noise', number_of_classes = 3, number_of_samples = number_of_samples, epochs = epochs, getData = getNoise)
    printResults(preds, [0, 0, 1, 1, 2, 2])
    
def trainForLaughter(number_of_samples=5, epochs=5):  
    #print('training on laughter and not laughter')
    (features, labels) = getLaughTracks(shuf=False, number_of_samples = 2, log=False)
    preds = trainAndSaveAndPredict(features, 'audio', number_of_classes = 2, number_of_samples = number_of_samples, epochs = epochs, getData = getLaughTracks, log = False)
    printResults(preds, [0, 0, 1, 1])


In [9]:
#trainForLaughter(number_of_samples=1, epochs=1)

In [10]:
#for i in range (0, 30):
#    number_of_samples = 1006 + (i * 1)
#    trainForLaughter(number_of_samples=number_of_samples, epochs=1)

In [11]:
trainForLaughter(number_of_samples=100, epochs=20)

model_name audio_100
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.738319
saving model ./model/audio_100/1_20/model
Step 2: loss 0.703907
saving model ./model/audio_100/2_20/model
Step 3: loss 0.680426
saving model ./model/audio_100/3_20/model
Step 4: loss 0.686973
saving model ./model/audio_100/4_20/model
Step 5: loss 0.671367
saving model ./model/audio_100/5_20/model
Step 6: loss 0.670204
saving model ./model/audio_100/6_20/model
Step 7: loss 0.656642
saving model ./model/audio_100/7_20/model
Step 8: loss 0.610177
saving model ./model/audio_100/8_20/model
Step 9: loss 0.637092
saving model ./model/audio_100/9_20/model
Step 10: loss 0.59811
saving model ./model/audio_100/10_20/model
Step 11: loss 0.596985
saving model ./model/audio_100/11_20/model
Step 12: loss 0.613042
saving model ./model/audio_100/12_20/model
Step 13: loss 0.590328
saving model ./model/audio_100/13_20/model
Step 14: loss 0.554993
saving model ./model/audio_100/14_20/model
Step 15: los

In [13]:
trainForLaughter(number_of_samples=201, epochs=40)

model_name audio_201
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.732376
saving model ./model/audio_201/1_40/model
Step 2: loss 0.633559
saving model ./model/audio_201/2_40/model
Step 3: loss 0.593709
saving model ./model/audio_201/3_40/model
Step 4: loss 0.572072
saving model ./model/audio_201/4_40/model
Step 5: loss 0.68271
saving model ./model/audio_201/5_40/model
Step 6: loss 0.595793
saving model ./model/audio_201/6_40/model
Step 7: loss 0.596696
saving model ./model/audio_201/7_40/model
Step 8: loss 0.556086
saving model ./model/audio_201/8_40/model
Step 9: loss 0.637458
saving model ./model/audio_201/9_40/model
Step 10: loss 0.590949
saving model ./model/audio_201/10_40/model
Step 11: loss 0.663197
saving model ./model/audio_201/11_40/model
Step 12: loss 0.565656
saving model ./model/audio_201/12_40/model
Step 13: loss 0.570229
saving model ./model/audio_201/13_40/model
Step 14: loss 0.843573
saving model ./model/audio_201/14_40/model
Step 15: los

In [15]:
(features, labels) = getLaughTracks(shuf=False, number_of_samples = 10, log=False)
printResults(predict(getModel('%s' % ('audio_201')), 2, features))

loading ./model/audio_201/40_40/model
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/audio_201/40_40/model
[[0.4287952  0.7107146 ]
 [0.7784004  0.31745234]
 [0.7917802  0.2827743 ]
 [0.80329025 0.26296672]
 [0.7596626  0.34796327]
 [0.8183522  0.26691192]
 [0.9127813  0.16164927]
 [0.9300833  0.13568938]
 [0.87669116 0.19700894]
 [0.90420705 0.16216436]
 [0.8533493  0.19718693]
 [0.39970633 0.5429855 ]
 [0.3739197  0.6703356 ]
 [0.179904   0.7579397 ]
 [0.28311822 0.7237356 ]
 [0.1798182  0.8356218 ]
 [0.16748893 0.7691494 ]
 [0.17559975 0.72274864]
 [0.3164792  0.6167977 ]
 [0.24945058 0.663799  ]]
[1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
expected results None


In [16]:
trainForLaughter(number_of_samples=500, epochs=100)

model_name audio_500
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.739929
saving model ./model/audio_500/1_100/model
Step 2: loss 0.684431
saving model ./model/audio_500/2_100/model
Step 3: loss 0.71865
saving model ./model/audio_500/3_100/model
Step 4: loss 0.685007
saving model ./model/audio_500/4_100/model
Step 5: loss 0.673483
saving model ./model/audio_500/5_100/model
Step 6: loss 0.667548
saving model ./model/audio_500/6_100/model
Step 7: loss 0.648605
saving model ./model/audio_500/7_100/model
Step 8: loss 0.626448
saving model ./model/audio_500/8_100/model
Step 9: loss 0.665971
saving model ./model/audio_500/9_100/model
Step 10: loss 0.669842
saving model ./model/audio_500/10_100/model
Step 11: loss 0.616114
saving model ./model/audio_500/11_100/model
Step 12: loss 0.66934
saving model ./model/audio_500/12_100/model
Step 13: loss 0.632326
saving model ./model/audio_500/13_100/model
Step 14: loss 0.622553
saving model ./model/audio_500/14_100/model

In [17]:
trainForLaughter(number_of_samples=600, epochs=300)

model_name audio_600
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
Step 1: loss 0.717405
saving model ./model/audio_600/1_300/model
Step 2: loss 0.694363
saving model ./model/audio_600/2_300/model
Step 3: loss 0.670209
saving model ./model/audio_600/3_300/model
Step 4: loss 0.65375
saving model ./model/audio_600/4_300/model
Step 5: loss 0.656008
saving model ./model/audio_600/5_300/model
Step 6: loss 0.63023
saving model ./model/audio_600/6_300/model
Step 7: loss 0.638339
saving model ./model/audio_600/7_300/model
Step 8: loss 0.653481
saving model ./model/audio_600/8_300/model
Step 9: loss 0.603731
saving model ./model/audio_600/9_300/model
Step 10: loss 0.609336
saving model ./model/audio_600/10_300/model
Step 11: loss 0.608169
saving model ./model/audio_600/11_300/model
Step 12: loss 0.664959
saving model ./model/audio_600/12_300/model
Step 13: loss 0.60818
saving model ./model/audio_600/13_300/model
Step 14: loss 0.560694
saving model ./model/audio_600/14_300/model


Step 122: loss 0.286293
saving model ./model/audio_600/122_300/model
Step 123: loss 0.207651
saving model ./model/audio_600/123_300/model
Step 124: loss 0.146712
saving model ./model/audio_600/124_300/model
Step 125: loss 0.20582
saving model ./model/audio_600/125_300/model
Step 126: loss 0.180919
saving model ./model/audio_600/126_300/model
Step 127: loss 0.134022
saving model ./model/audio_600/127_300/model
Step 128: loss 0.290647
saving model ./model/audio_600/128_300/model
Step 129: loss 0.277411
saving model ./model/audio_600/129_300/model
Step 130: loss 0.175952
saving model ./model/audio_600/130_300/model
Step 131: loss 0.214518
saving model ./model/audio_600/131_300/model
Step 132: loss 0.181963
saving model ./model/audio_600/132_300/model
Step 133: loss 0.175504
saving model ./model/audio_600/133_300/model
Step 134: loss 0.216229
saving model ./model/audio_600/134_300/model
Step 135: loss 0.124865
saving model ./model/audio_600/135_300/model
Step 136: loss 0.329814
saving mode

Step 241: loss 0.169814
saving model ./model/audio_600/241_300/model
Step 242: loss 0.120375
saving model ./model/audio_600/242_300/model
Step 243: loss 0.146614
saving model ./model/audio_600/243_300/model
Step 244: loss 0.169205
saving model ./model/audio_600/244_300/model
Step 245: loss 0.194513
saving model ./model/audio_600/245_300/model
Step 246: loss 0.16926
saving model ./model/audio_600/246_300/model
Step 247: loss 0.124347
saving model ./model/audio_600/247_300/model
Step 248: loss 0.204761
saving model ./model/audio_600/248_300/model
Step 249: loss 0.0950304
saving model ./model/audio_600/249_300/model
Step 250: loss 0.129634
saving model ./model/audio_600/250_300/model
Step 251: loss 0.0899939
saving model ./model/audio_600/251_300/model
Step 252: loss 0.181625
saving model ./model/audio_600/252_300/model
Step 253: loss 0.105643
saving model ./model/audio_600/253_300/model
Step 254: loss 0.233315
saving model ./model/audio_600/254_300/model
Step 255: loss 0.139201
saving mo

In [19]:
(features, labels) = getLaughTracks(shuf=False, number_of_samples = 4, log=False)
preds = predict(getModel('%s' % ('audio_600')), 2, features)
printResults(preds, [0, 0, 0, 0, 1, 1, 1, 1])

loading ./model/audio_600/300_300/model
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/audio_600/300_300/model
[[0.93022645 0.04779049]
 [0.9882008  0.01123085]
 [0.99851245 0.00118623]
 [0.9982622  0.00138261]
 [0.8071062  0.1965018 ]
 [0.649517   0.34406373]
 [0.13383405 0.8822048 ]
 [0.21634802 0.8144273 ]]
[0 0 0 0 0 0 1 1]
expected results [0, 0, 0, 0, 1, 1, 1, 1]


In [21]:
(features, labels) = getLaughTracks(shuf=False, number_of_samples = 4, log=False)
preds = predict(getModel('%s' % ('audio_2000')), 2, features)
printResults(preds, [0, 0, 0, 0, 1, 1, 1, 1])

loading ./model/audio_2000/7/model
INFO:tensorflow:Restoring parameters from ./vggish_model.ckpt
INFO:tensorflow:Restoring parameters from ./model/audio_2000/7/model
[[0.46733063 0.72856736]
 [0.5200044  0.58841306]
 [0.5308451  0.5193135 ]
 [0.5482788  0.53807145]
 [0.48719636 0.55772114]
 [0.47477028 0.7101242 ]
 [0.46824598 0.74748474]
 [0.47193593 0.71191275]]
[1 1 0 0 1 1 1 1]
expected results [0, 0, 0, 0, 1, 1, 1, 1]
